In [1]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 46.7 MB/s eta 0:00:00


In [4]:
#@title Advanced Hand Drawing with Custom Colors in Colab (Optimized for Speed & GPU) {display-mode: "code"}
import cv2
import mediapipe as mp
import numpy as np
import time
from google.colab.patches import cv2_imshow
from IPython.display import clear_output

# --------------------- Helper Functions for Custom Drawing ---------------------
def get_finger_color(idx):
    """Return a BGR color based on the finger group for a given landmark index."""
    if idx in [1, 2, 3, 4]:
        return (0, 0, 255)      # Thumb: Red
    elif idx in [5, 6, 7, 8]:
        return (0, 255, 0)      # Index: Green
    elif idx in [9, 10, 11, 12]:
        return (255, 0, 0)      # Middle: Blue
    elif idx in [13, 14, 15, 16]:
        return (0, 255, 255)    # Ring: Yellow
    elif idx in [17, 18, 19, 20]:
        return (255, 0, 255)    # Pinky: Magenta
    else:
        return (255, 255, 255)  # Default: White

def get_landmark_color(idx):
    """For landmarks, use a special color for the wrist (idx==0) then same as finger color."""
    if idx == 0:
        return (0, 165, 255)    # Wrist: Orange
    else:
        return get_finger_color(idx)

# --------------------- Initialize MediaPipe & Video Capture ---------------------
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1,
                       min_detection_confidence=0.8,
                       min_tracking_confidence=0.8)

# Use your video file or webcam; here we use a video file.
cap = cv2.VideoCapture("/content/the gamet 12.13.22_5defac8f.mp4")
# For webcam, use: cap = cv2.VideoCapture(0)

canvas = None  # White canvas will be initialized once frame size is known

# Variables for drawing with index finger (landmark 8)
prev_x, prev_y = 0, 0
stroke_color = None  # A new random color will be assigned for each stroke

# Gesture-based canvas clear cooldown (in seconds); reduced for responsiveness
CLEAR_COOLDOWN = 0.2
last_clear_time = 0

print("Advanced Hand Drawing in Colab (Optimized for Speed & GPU):")
print(" - Drawing is done with the index fingertip (landmark 8).")
print(" - Each stroke gets a new random color.")
print(" - Hand landmarks are drawn with custom colors for each finger group.")
print(" - Bring index and middle fingertips together (~40 px apart) to clear the canvas.")
print(" - GPU acceleration is used if available (requires GPU-enabled OpenCV).")
print(" - Interrupt the cell execution to stop.")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or failed to capture frame. Exiting...")
            break

        # Resize frame to 640x480 for faster processing.
        frame = cv2.resize(frame, (640, 480))
        # Flip frame horizontally for a mirror view.
        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape

        # Initialize canvas (white) once we know the frame size.
        if canvas is None:
            canvas = np.ones_like(frame) * 255

        # Convert the frame to RGB as required by MediaPipe.
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        # --------------------- Draw Advanced Hand Landmarks ---------------------
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                # Draw custom-colored connections.
                for connection in mp_hands.HAND_CONNECTIONS:
                    start_idx, end_idx = connection
                    start_lm = handLms.landmark[start_idx]
                    end_lm = handLms.landmark[end_idx]
                    x1, y1 = int(start_lm.x * w), int(start_lm.y * h)
                    x2, y2 = int(end_lm.x * w), int(end_lm.y * h)
                    cv2.line(frame, (x1, y1), (x2, y2), get_finger_color(end_idx), 2)
                # Draw custom-colored landmarks.
                for idx, lm in enumerate(handLms.landmark):
                    x, y = int(lm.x * w), int(lm.y * h)
                    cv2.circle(frame, (x, y), 5, get_landmark_color(idx), cv2.FILLED)

                # --------------------- Draw on the Canvas with Index Fingertip ---------------------
                index_tip = handLms.landmark[8]
                ix, iy = int(index_tip.x * w), int(index_tip.y * h)
                middle_tip = handLms.landmark[12]
                mx, my = int(middle_tip.x * w), int(middle_tip.y * h)

                # Clear canvas gesture: if index and middle fingertips are close.
                if np.hypot(ix - mx, iy - my) < 40 and (time.time() - last_clear_time) > CLEAR_COOLDOWN:
                    canvas = np.ones_like(frame) * 255  # Reset canvas to white.
                    prev_x, prev_y = 0, 0
                    stroke_color = None
                    last_clear_time = time.time()

                # If starting a new stroke, initialize the starting point and assign a new color.
                if prev_x == 0 and prev_y == 0:
                    prev_x, prev_y = ix, iy
                    if stroke_color is None:
                        stroke_color = tuple(np.random.randint(0, 256, 3).tolist())
                else:
                    # Draw a line from the previous point to the current index fingertip position.
                    cv2.line(canvas, (prev_x, prev_y), (ix, iy), stroke_color, 5, cv2.LINE_AA)
                    prev_x, prev_y = ix, iy
        else:
            # Reset stroke variables if no hand is detected.
            prev_x, prev_y = 0, 0
            stroke_color = None

        # --------------------- Combine and Display Panels ---------------------
        # Add labels to the panels.
        cv2.putText(frame, "Advanced Hand Landmarks", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.putText(canvas, "Drawing Canvas", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
        # Stack the landmark view and drawing canvas side by side.
        combined = np.hstack((frame, canvas))
        cv2_imshow(combined)
        clear_output(wait=True)

except KeyboardInterrupt:
    print("Interrupted by user. Exiting...")

finally:
    cap.release()
    cv2.destroyAllWindows()


End of video or failed to capture frame. Exiting...


In [8]:
#@title Advanced Hand Drawing: Fast Output Only (No Video Save) {display-mode: "code"}
import cv2
import mediapipe as mp
import numpy as np
import time
from google.colab.patches import cv2_imshow
from IPython.display import clear_output

# --------------------- Helper Functions for Custom Drawing ---------------------
def get_finger_color(idx):
    """Return a BGR color based on the finger group for a given landmark index."""
    if idx in [1, 2, 3, 4]:
        return (0, 0, 255)      # Thumb: Red
    elif idx in [5, 6, 7, 8]:
        return (0, 255, 0)      # Index: Green
    elif idx in [9, 10, 11, 12]:
        return (255, 0, 0)      # Middle: Blue
    elif idx in [13, 14, 15, 16]:
        return (0, 255, 255)    # Ring: Yellow
    elif idx in [17, 18, 19, 20]:
        return (255, 0, 255)    # Pinky: Magenta
    else:
        return (255, 255, 255)  # Default: White

def get_landmark_color(idx):
    """Return a special color for the wrist (idx==0) then the finger color."""
    if idx == 0:
        return (0, 165, 255)    # Wrist: Orange
    else:
        return get_finger_color(idx)

# --------------------- Initialize MediaPipe & Video Capture ---------------------
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)

# Use your video file or webcam. Here we use a video file.
cap = cv2.VideoCapture("/content/the gamet 12.13.22_5defac8f.mp4")
# For webcam, uncomment the next line:
# cap = cv2.VideoCapture(0)

canvas = None                # White drawing canvas (initialized once frame size is known)
prev_x, prev_y = 0, 0        # For drawing with the index fingertip
stroke_color = None          # A new random color per stroke

# Gesture-based canvas clear cooldown (seconds)
CLEAR_COOLDOWN = 0.2
last_clear_time = 0

print("Processing video FAST—displaying output only (no saving video).")
print("Press Ctrl+C or interrupt the cell to stop.")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or capture error.")
            break

        # Downscale frame for speed: 320x240 resolution.
        frame = cv2.resize(frame, (320, 240))
        frame = cv2.flip(frame, 1)  # Mirror view.
        h, w, _ = frame.shape

        # Initialize the white canvas if needed.
        if canvas is None:
            canvas = np.ones_like(frame) * 255

        # Convert the frame to RGB (MediaPipe requires RGB images).
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        # --------------------- Draw Advanced Hand Landmarks & Draw on Canvas ---------------------
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                # Draw connections.
                for connection in mp_hands.HAND_CONNECTIONS:
                    start_idx, end_idx = connection
                    start_lm = handLms.landmark[start_idx]
                    end_lm = handLms.landmark[end_idx]
                    x1, y1 = int(start_lm.x * w), int(start_lm.y * h)
                    x2, y2 = int(end_lm.x * w), int(end_lm.y * h)
                    cv2.line(frame, (x1, y1), (x2, y2), get_finger_color(end_idx), 1)
                # Draw landmarks.
                for idx, lm in enumerate(handLms.landmark):
                    x, y = int(lm.x * w), int(lm.y * h)
                    cv2.circle(frame, (x, y), 3, get_landmark_color(idx), cv2.FILLED)

                # Use index fingertip (landmark 8) for drawing.
                index_tip = handLms.landmark[8]
                ix, iy = int(index_tip.x * w), int(index_tip.y * h)
                # Use middle fingertip (landmark 12) for canvas-clear gesture.
                middle_tip = handLms.landmark[12]
                mx, my = int(middle_tip.x * w), int(middle_tip.y * h)

                # Clear canvas if index and middle fingertips are close.
                if np.hypot(ix - mx, iy - my) < 40 and (time.time() - last_clear_time) > CLEAR_COOLDOWN:
                    canvas = np.ones_like(frame) * 255
                    prev_x, prev_y = 0, 0
                    stroke_color = None
                    last_clear_time = time.time()

                # Start a new stroke if necessary.
                if prev_x == 0 and prev_y == 0:
                    prev_x, prev_y = ix, iy
                    if stroke_color is None:
                        stroke_color = tuple(np.random.randint(0, 256, 3).tolist())
                else:
                    # Draw a line on the canvas.
                    cv2.line(canvas, (prev_x, prev_y), (ix, iy), stroke_color, 3, cv2.LINE_AA)
                    prev_x, prev_y = ix, iy
        else:
            # Reset stroke if no hand is detected.
            prev_x, prev_y = 0, 0
            stroke_color = None

        # --------------------- Combine Panels & Display ---------------------
        cv2.putText(frame, "Hand Landmarks", (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.putText(canvas, "Drawing Canvas", (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        combined = np.hstack((frame, canvas))
        cv2_imshow(combined)
        clear_output(wait=True)

except KeyboardInterrupt:
    print("Processing interrupted by user.")

finally:
    cap.release()
    cv2.destroyAllWindows()


End of video or capture error.


In [14]:
#@title Advanced Hand Drawing: Fast Output Only (Continuous Connected Drawing) {display-mode: "code"}
import cv2
import mediapipe as mp
import numpy as np
import time
from google.colab.patches import cv2_imshow
from IPython.display import clear_output

# --------------------- Helper Functions for Custom Drawing ---------------------
def get_finger_color(idx):
    """Return a BGR color based on the finger group for a given landmark index."""
    if idx in [1, 2, 3, 4]:
        return (0, 0, 255)      # Thumb: Red
    elif idx in [5, 6, 7, 8]:
        return (0, 255, 0)      # Index: Green
    elif idx in [9, 10, 11, 12]:
        return (255, 0, 0)      # Middle: Blue
    elif idx in [13, 14, 15, 16]:
        return (0, 255, 255)    # Ring: Yellow
    elif idx in [17, 18, 19, 20]:
        return (255, 0, 255)    # Pinky: Magenta
    else:
        return (255, 255, 255)  # Default: White

def get_landmark_color(idx):
    """Return a special color for the wrist (idx==0) then the finger color."""
    if idx == 0:
        return (0, 165, 255)    # Wrist: Orange
    else:
        return get_finger_color(idx)

# --------------------- Initialize MediaPipe & Video Capture ---------------------
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)

# Use your video file or webcam. Here we use a video file.
cap = cv2.VideoCapture("/content/yarab53b67.mp4")
# For webcam, uncomment the next line:
# cap = cv2.VideoCapture(0)

canvas = None                # White drawing canvas (initialized once frame size is known)
prev_x, prev_y = 0, 0        # For drawing with the index fingertip
stroke_color = None          # A new random color per stroke

# Gesture-based clear cooldown (unused now, canvas remains permanent)
CLEAR_COOLDOWN = 0.2
last_clear_time = 0

print("Processing video FAST—displaying output only (continuous connected drawing).")
print("Press Ctrl+C or interrupt the cell to stop.")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or capture error.")
            break

        # Downscale frame for speed: 320x240 resolution.
        frame = cv2.resize(frame, (320, 240))
        frame = cv2.flip(frame, 1)  # Mirror view.
        h, w, _ = frame.shape

        # Initialize the white canvas if needed.
        if canvas is None:
            canvas = np.ones_like(frame) * 255

        # Convert frame to RGB (MediaPipe requires RGB).
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        # --------------------- Draw Advanced Hand Landmarks & Continuous Drawing ---------------------
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                # Draw hand connections.
                for connection in mp_hands.HAND_CONNECTIONS:
                    start_idx, end_idx = connection
                    start_lm = handLms.landmark[start_idx]
                    end_lm = handLms.landmark[end_idx]
                    x1, y1 = int(start_lm.x * w), int(start_lm.y * h)
                    x2, y2 = int(end_lm.x * w), int(end_lm.y * h)
                    cv2.line(frame, (x1, y1), (x2, y2), get_finger_color(end_idx), 1)
                # Draw hand landmarks.
                for idx, lm in enumerate(handLms.landmark):
                    x, y = int(lm.x * w), int(lm.y * h)
                    cv2.circle(frame, (x, y), 3, get_landmark_color(idx), cv2.FILLED)

                # --------------------- Continuous Drawing on Canvas ---------------------
                index_tip = handLms.landmark[8]
                ix, iy = int(index_tip.x * w), int(index_tip.y * h)
                # (Optional) Use middle fingertip for gesture detection; clearing is disabled.
                middle_tip = handLms.landmark[12]
                mx, my = int(middle_tip.x * w), int(middle_tip.y * h)

                # (Clear canvas gesture code removed so that drawn lines remain continuous.)

                # If this is the very first detection, initialize the starting point.
                if prev_x == 0 and prev_y == 0:
                    prev_x, prev_y = ix, iy
                    if stroke_color is None:
                        stroke_color = tuple(np.random.randint(0, 256, 3).tolist())
                else:
                    # Draw a line connecting the previous point to the current index fingertip.
                    cv2.line(canvas, (prev_x, prev_y), (ix, iy), stroke_color, 3, cv2.LINE_AA)
                    prev_x, prev_y = ix, iy
        # If no hand is detected, DO NOT reset prev_x, prev_y (keep drawing connected).
        # This will connect the last known point with the next detected point.

        # --------------------- Combine Panels & Display ---------------------
        cv2.putText(frame, "Hand Landmarks", (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.putText(canvas, "Drawing Canvas", (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        combined = np.hstack((frame, canvas))
        cv2_imshow(combined)
        clear_output(wait=True)

except KeyboardInterrupt:
    print("Processing interrupted by user.")

finally:
    cap.release()
    cv2.destroyAllWindows()


End of video or capture error.
